# Раздел 6. Уравнения с частными производными параболического типа.
## Лабораторная работа №6. Методы решения квазилинейного уравнения теплопроводности.
### Вариант 2 Задание 5
_Яромир Водзяновский_

Дифференциальная задача

$$\frac{\partial u}{\partial t} = \frac{\partial}{\partial x} \left( u^{1/3} \frac{\partial u}{\partial x} \right) + \frac{\partial}{\partial y} \left( u^{1/3} \frac{\partial u}{\partial y} \right), \;\;\; 0 < t\leq 1, \; 0 < x,y <1    $$
$$u(0,x,y) = (1+x+y)^6 / 27000, \; 0 \leq x,y \leq 1  $$
$$u(t,0,y) = (1+y)^6 / (30-28t)^3, \; 0 < t \leq 1, \; 0 \leq y \leq 1  $$
$$u(t,1,y) = (2+y)^6/(30-28t)^3, \; 0 < t \leq 1, \; 0 \leq y \leq 1   $$
$$u(t,x,0) = (1+x)^6 /(30-28t)^3, \; 0 < t \leq 1, \; 0 < x < 1    $$
$$u(t,x,1) = (2+x)^6 /(30-28t)^3, \; 0 < t \leq 1, \; 0 < x < 1    $$
Решение в виде: 
$$u = (C_z + x + y)^{2/\mu} \left[ C_t - \frac{4 (\mu +2)}{\mu}t \right]^{-1/\mu} $$

Для нашей задачи:
$$C_t = 30, \; C_z = 1,\; \mu = \frac{1}{3}$$

In [4]:
import numpy as np

In [21]:
def anal(x,y):
    return (C_z + x+y)**(2/mu) * (C_t - 4*(mu+2)/mu )**(-1/mu)


In [35]:
def U_anal(x,y):   

    y_a = []
    for i in range(K+1):
        for j in range(L+1):
            y_a.append(y[i])
    y_a = np.array(y_a)

    x_a = np.array([])
    for i in range(K+1):
        x_a = np.concatenate((x_a,x))

    U_an = np.vectorize(anal)(x_a,y_a).reshape(L+1,K+1)
    return U_an

In [54]:
C_z = 1
C_t = 30
mu = 1/3
N = 3
K = 4
L = 5
tau = 1/N
h_x = 1/L
h_y = 1/K


In [55]:
x = np.arange(0,1+1/(L*1000) , 1/L)
y = np.arange(0,1+1/(K*1000) , 1/K)
t = np.ones((1,N+1))[0]

In [56]:
U_anal(x,y)

array([[ 0.125     ,  0.373248  ,  0.941192  ,  2.097152  ,  4.251528  ],
       [ 8.        ,  0.47683716,  1.1617643 ,  2.52239838,  5.01118439],
       [ 9.27754735, 16.21829224,  1.42382813,  3.01719613,  5.88073513],
       [10.72076513, 18.50448613, 30.51757813,  3.59036255,  6.87254641],
       [12.34641221, 21.05315494, 34.36774961, 54.06375122,  8.        ],
       [14.172488  , 23.887872  , 38.614472  , 60.236288  , 91.125     ]])

In [57]:
U = np.zeros((N,L,K))
U

array([[[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       [[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       [[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]]])

In [ ]:
a_l = -((U[n][l+1][k])**mu + (U[n][l][k])**mu)*tau/(2*h_x**2)
c_l = -((U[n][l][k])**mu + (u[n][l-1][k])**mu)*tau/(2*h_x**2)
b_l = 1-a_l-c_l